<a href="https://colab.research.google.com/github/jcval94/Proyecto_Modulo2/blob/main/Encuesta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Funciones para conectarse a Google Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
from oauth2client.client import GoogleCredentials
import gspread_dataframe as gd
from google.auth import default

In [3]:
from sklearn.cluster import KMeans
# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import style
import seaborn as sns
style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado
# ==============================================================================
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

In [4]:
import pickle as pk
import zipfile

In [5]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
creds, _ = default()
gc = gspread.authorize(creds)
pd.options.mode.chained_assignment = None

In [7]:
# Cargar modelos pre-entrenados

url = 'https://drive.google.com/file/d/1ofGl8U20ObpNef1hTdFOzxaQIHcfSUX4/view?usp=sharing'
id_ = url.split('/')[-2]

# Toco la puerta para ver si hay alguien
links = drive.CreateFile({'id':id_})

# Si hay alguien (un zip) lo descomprimo para ver su interior
links.GetContentFile('folder.zip',mimetype=links['mimeType'])
with zipfile.ZipFile('folder.zip',"r") as zip_ref:
  zip_ref.extractall()

In [ ]:
# Cargar información de encuestas completa
cuaderno = 'https://docs.google.com/spreadsheets/d/1B5nKjbt9Pw_vFjYU3rufyUACpPQDkO6DPmNDNUmDRso/edit?usp=sharing'
  
Estudiantes = ["Respuestas de formulario 1"]

ws = gc.open('Información de contacto (Respuestas)')

# print('Calificando...')
for estudiante in Estudiantes:
  existing_ = gd.get_as_dataframe(ws.worksheet(estudiante))
  existing = existing_.copy()

NameError: ignored

In [ ]:
#Elimino filas y columnas que estén completamente vacías
existing.dropna(how='all', inplace=True)
existing.dropna(how='all', axis='columns', inplace=True)

In [ ]:
mucho_texto = 'Coloca la letra adecuada o letras (si piensas que está entre esos puntos), ¬F significa que se excluye la observación F y P significa percentil, (si no sabes F como dicen los chavos) '

In [ ]:
existing.columns = [x.replace(mucho_texto,'') for x in existing.columns]

id_cols = ['Marca temporal', 'Nombre', 'Correo electrónico','Contraseña']

progra_cols = ['¿De qué lenguajes de programación has escuchado?','Puedo imprimir "Hola mundo" en los siguientes lenguajes:']
gustos_cols = ['He visto los siguientes shows',
       'Tengo una cuenta activa en las siguientes redes:',
       'En general qué consideras más importante']

mate_cols = ['Sin hacer trampa, cuánto es',
       '¿Si tengo 110 manzanas, le di X a mi primo y ahora si resto las mías menos las suyas me da 100 cuántas manzanas le di?',
       'a y b son números enteros y sabemos qué']

estad_cols = [x for x in existing.columns if x not in id_cols+progra_cols+gustos_cols+mate_cols]

In [ ]:
existing_program1 = existing[progra_cols[0]].str.get_dummies(sep=',').add_prefix('know_')
existing_program1['no_leng_k'] = existing_program1.sum(axis=1)
existing_program2 = existing[progra_cols[1]].str.get_dummies(sep=',').add_prefix('print_')
existing_program2['no_leng_p'] = existing_program2.sum(axis=1)

existing_program = pd.concat([existing_program1,existing_program2], axis=1)

In [ ]:
gpo = existing[gustos_cols[0]].str.get_dummies(sep=',').add_prefix('Show_')
gpo['Show_tot'] = gpo.sum(axis=1)
gpo1 = existing[gustos_cols[1]].str.get_dummies(sep=',').add_prefix('Red_')
gpo1['Red_tot'] = gpo.sum(axis=1)
gpo2 = existing[gustos_cols[2]].str.get_dummies(sep=',').add_prefix('Gpo_')

gpo = pd.concat([gpo,gpo1], axis =1)
gpo = pd.concat([gpo,gpo2], axis =1)

In [ ]:
mate = existing[mate_cols]
mate.loc[:,'expo_S'] = (mate[mate_cols[0]]==-81).astype(int)
mate.loc[:,'expo_no_sabe'] = (mate[mate_cols[0]]=='Hmmmmm...').astype(int)
mate.loc[:,'expo_N'] = (~mate[mate_cols[0]].isin([-81,'Hmmmmm...'])).astype(int)

mate.loc[:,'dif_S'] = (existing[mate_cols[1]]==5).astype(int)
mate.loc[:,'dif_no_sabe'] = (existing[mate_cols[1]]=='Ehhhhh... Ehhhh...').astype(int)
mate.loc[:,'dif_N'] = (~existing[mate_cols[1]].isin([5,'Ehhhhh... Ehhhh...'])).astype(int)

mate.loc[:,'mayor_S'] = (existing[mate_cols[2]]=='No se puede determinar con la información dada').astype(int)
mate.loc[:,'mayor_no_sabe'] = (existing[mate_cols[2]]=='Un gatito').astype(int)
mate.loc[:,'mayor_N'] = (~existing[mate_cols[2]].isin(['No se puede determinar con la información dada','Un gatito'])).astype(int)

In [ ]:
mate['calif_m']= [a+b+c for a, b, c in mate[['expo_S','dif_S','mayor_S']].values]

In [ ]:
mate_df = mate[[col for col in mate.columns if col not in mate_cols]]

In [ ]:
stat = existing[estad_cols[1:-1]]

resp = ['E','D','D','D','D','D','C','C','C','C']
nms_s = []
for key, res in zip(estad_cols[1:-1], resp):
  nam_col_s = 'S_'+key.replace(']','').replace('[','')
  stat[nam_col_s] = stat[key].apply(lambda x: res in str(x).split(',')).astype(int)
  nms_s.append(nam_col_s)

stat['calif_s'] = stat[nms_s].sum(axis=1)

# Deben ser iguales, entre más cerca del 1 mejor
stat['C_C1'] = (stat['[Cuantil 1 (¬F)]']==stat['[Cuantil 1]']).astype(int)
# Deben ser distintos, entre más cerca del 1 mejor
stat['C_Me'] = (stat['[Media (¬F)]']!=stat['[Media]']).astype(int)
# Deben ser iguales, entre más cerca del 1 mejor
stat['C_Md'] = (stat['[Mediana (¬F)]']==stat['[Mediana]']).astype(int)
# Deben ser iguales, entre más cerca del 1 mejor
stat['C_Mo'] = (stat['[Moda (¬F)]']==stat['[Moda]']).astype(int)
# Deben ser iguales, entre más cerca del 1 mejor
stat['C_Pe'] = (stat['[P 26.22 (¬F)]']==stat['[P 26.22]']).astype(int)

# Deben ser distintos, entre más cerca del 1 mejor
stat['C_PC'] = (stat['C_Pe']==stat['C_C1']).astype(int)
# Deben ser distintos, entre más cerca del 1 mejor
stat['C_ALL'] = (existing[estad_cols[1:-2]].nunique(axis=1) != 1).astype(int)

stat = stat[[col for col in stat.columns if col not in estad_cols[1:-1]]]

stat.loc[:,'entiende_grafica'] = (existing[estad_cols[0]] == 'Sé lo que estoy viendo').astype(int)

In [ ]:
# dfs = stat, mate, gpo, existing_program

In [ ]:
pca_pipe = make_pipeline(StandardScaler(), PCA())
km_modelo = KMeans(init="k-means++", n_clusters=3, n_init=3, random_state=324)

In [ ]:
def get_pca(stat_=stat, pca_model=pca_pipe, entrenar=False):
  
  if entrenar:
    pca_model.fit(stat_)

  modelo_pca = pca_model.named_steps['pca']
  proyecciones = pca_model.transform(X=stat_)
  proyecciones = pd.DataFrame(
      proyecciones,
      columns = ['PC'+str(n+1) for n in range(len(stat_.columns))],
      index   = stat_.index
  )

  pcas_val = ['PC'+str(i+1) for i, v in enumerate(np.cumsum(modelo_pca.explained_variance_ratio_)<.9) if v]
  PCA_df = proyecciones[pcas_val]

  return PCA_df, pca_model

In [ ]:
def get_kmn(stat_, PCA_df,plot_km, km_model, entrenar = False):
  if entrenar:
    db = km_model.fit(PCA_df)
    stat_['cluster'] = db.labels_
  else:
    db = km_model
    stat_['cluster'] = km_model.predict(PCA_df)

  if plot_km:
    colors = ['red', 'blue', 'purple', 'green','orange', 'black', 'gray']
    nclus = np.unique(stat_['cluster'])
    ax = sns.scatterplot(PCA_df['PC1'], PCA_df['PC2'], hue=stat_['cluster'], palette=colors[:len(nclus)], alpha=0.5, s=7)

    means = np.vstack([PCA_df[stat_['cluster']== i].mean(axis=0) for i in range(len(nclus))])
    ax = sns.scatterplot(means[:, 0], means[:, 1], hue=range(len(nclus)),
                        palette=colors[:len(nclus)], s=20, ec='black', legend=False, ax=ax)
    plt.show()

  return stat_, db

# Estadística

In [ ]:
# PCA_df_, pca_ajustado = get_pca(stat,pca_pipe,True)
# stat, km_ajustado = get_kmn(stat, PCA_df_, True, km_modelo,True)

# #Guardar modelo entrenado
# pk.dump(pca_ajustado, open("pca_stat.pkl","wb"))
# pk.dump(km_ajustado, open("km_stat.pkl","wb"))

In [ ]:
#Leer modelo precargado
pca_reload = pk.load(open("pca_stat.pkl",'rb'))
km_reload = pk.load(open("km_stat.pkl",'rb'))

PCA_df_, pca_ajustado = get_pca(stat,pca_reload)
stat, km_ajustado = get_kmn(stat, PCA_df_, True,km_reload,False)

In [ ]:
pd.pivot_table(stat, index='cluster')

In [ ]:
existing['calif_s'] = stat['calif_s']
existing['cluster_s'] = stat['cluster']
existing.iloc[stat[stat['calif_s'] == max(stat['calif_s'])].index]

# Matemáticas

In [ ]:
# PCA_df_, pca_ajustado = get_pca(mate_df,pca_pipe,True)
# mate_df, km_ajustado = get_kmn(mate_df, PCA_df_, True, km_modelo,True)

# #Guardar modelo entrenado
# pk.dump(pca_ajustado, open("pca_mate.pkl","wb"))
# pk.dump(km_ajustado, open("km_mate.pkl","wb"))

In [ ]:
#Leer modelo precargado
pca_reload = pk.load(open("pca_mate.pkl",'rb'))
km_reload = pk.load(open("km_mate.pkl",'rb'))

PCA_df_, pca_ajustado = get_pca(mate_df,pca_reload)
mate_df, km_ajustado = get_kmn(mate_df, PCA_df_, True,km_reload,False)

In [ ]:
pd.pivot_table(mate_df, index='cluster')

In [ ]:
existing['calif_m'] = mate_df['calif_m']
existing['cluster_m'] = mate_df['cluster']

In [ ]:
existing.iloc[mate_df[mate_df['calif_m'] == max(mate_df['calif_m'])].index]

# Programación

In [ ]:
sum_cols = existing_program.sum(axis=0)
col_c = sum_cols[sum_cols>int(((len(existing_program.columns)-2)/2)*.1)-1]

progra = existing_program[col_c.index]

In [ ]:
# PCA_df_, pca_ajustado = get_pca(progra,pca_pipe,True)
# progra, km_ajustado = get_kmn(progra, PCA_df_, True, km_modelo,True)

# #Guardar modelo entrenado
# pk.dump(pca_ajustado, open("pca_progra.pkl","wb"))
# pk.dump(km_ajustado, open("km_progra.pkl","wb"))

In [ ]:
#Leer modelo precargado
pca_reload = pk.load(open("pca_progra.pkl",'rb'))
km_reload = pk.load(open("km_progra.pkl",'rb'))

PCA_df_, pca_ajustado = get_pca(progra,pca_reload)
progra, km_ajustado = get_kmn(progra, PCA_df_, True,km_reload,False)

In [ ]:
pd.pivot_table(progra, index='cluster')

In [ ]:
existing['cluster_p'] = progra['cluster']

# Gustos

In [ ]:
sum_cols = gpo.sum(axis=0)
col_c = sum_cols[sum_cols>int(((len(gpo.columns)-2)/2)*.1)-1]

In [ ]:
# gustos = gpo[[cl for cl in col_c.index if not ("Red_" in cl or "Gpo_" in cl)]]

# PCA_df_ = get_pca(gustos)
# gustos = get_kmn(gustos, PCA_df_, True)

In [ ]:
calif_clus = existing[['calif_m','calif_s','cluster_m','cluster_s','cluster_p']]

In [ ]:
ax = sns.jointplot(
    data=calif_clus,
    x="calif_m", y="calif_s", hue="cluster_m",
    # kind = 'hex',
    color = 'g'
)

In [ ]:
ax = sns.jointplot(
    data=calif_clus,
    x="calif_m", y="calif_s", hue="cluster_s",
    # kind = 'hex',
    color = 'g'
)

# Los 3 vecinos más cercanos

In [ ]:
prof_cols = ['Marca temporal', 'Nombre', 'Correo electrónico', 'Contraseña']
Agrupamiento = existing[['cluster_s', 'cluster_m', 'cluster_p']]

In [ ]:
km_modelo = KMeans(init="k-means++", n_clusters=3, n_init=3, random_state=324)
model = km_modelo.fit(Agrupamiento)
Agrupamiento['CL'] = model.labels_
existing['CL_conocimientos'] = model.labels_

In [ ]:
km_modelo = KMeans(init="k-means++", n_clusters=3, n_init=3, random_state=324)
model_gpo = km_modelo.fit(gpo)
existing['CL_gustos'] = model_gpo.labels_

In [ ]:
existing['No_leng'] = progra['no_leng_p']

In [ ]:
cols_a = ['Nombre', 'Correo electrónico','calif_m','calif_s','No_leng','CL_conocimientos','CL_gustos','En general qué consideras más importante']
imprimir = existing[cols_a].sort_values(['CL_conocimientos','CL_gustos'])
imprimir.to_csv('Resultados_Encuesta.csv',index=False)

In [ ]:
imprimir

# Filtrar alumnos

In [ ]:
alumnos = '/content/G15-alumnos - Sheet1.csv'
df_alm = pd.read_csv(alumnos, header=None)

In [ ]:
!pip install unidecode

In [ ]:
nombres = df_alm[0].apply(lambda x: unidecode.unidecode(str(x).lower()).split(' '))
NBS = list(grup['Nombre'].apply(lambda x: unidecode.unidecode(str(x).lower())))

In [ ]:
guardado = {}
for nombre in NBS:
  catal = {}
  for i, n in enumerate(nombres):
    conta = 0
    for ap in n:
      if ap in nombre:
        conta +=1
    catal[i] = conta

  aplica = [(key, val) for key, val in catal.items() if val>0]
  guardado[nombre] = aplica

In [ ]:
coincidencias = {key: v for key, v in guardado.items() if len(v)>0}

In [ ]:
registrado = []
coin_100 = []
for key, v in coincidencias.items():
  nombres_full = len(key.split(' '))
  coincidencia_100 = [a for a, b in v if b==nombres_full]
  
  if len(coincidencia_100)>0:
    registrado.append([key, True])
    coin_100.append(key)
  else:
    diferencias = [abs(b-nombres_full) for a, b in v]
    # si al menos un registro es diferente
    if not all([d==diferencias[0] for d in diferencias]):
      carcano = v[argmin([abs(b-nombres_full) for a, b in v])][0]
      registrado.append([key, nombres[carcano]])

  

In [ ]:
coin_100

['airin ishbel saavedra flores',
 'cindy guadalupe rosendo castro',
 'axel castro',
 'kevin ruy roldan amador',
 'giselle altagrassi herrera nunez',
 'angel brito segura',
 'maria azucena peredo aguilar',
 'victor armando martinez cueto',
 'jonatan andrade',
 'marianne dennise rios payro',
 'kareem gomez',
 'jonathan castillo',
 'jahir antonio rodriguez perales',
 'citlali gil aguillon',
 'aline yadira rodriguez becerra',
 'vanessa',
 'iker ofir mejia meza',
 'alvaro',
 'aridai gonzalez serralde',
 'cristian axel cortes perez',
 'arturo benjamin lopez becerril',
 'dayana ailed jimenez copeno',
 'estefania chavez huerta',
 'evelyn solano',
 'diego',
 'miguel',
 'flores alonso oscar david',
 'giselle alejandra gonzalez lopez',
 'hannia rojas',
 'joseph fabrizio trejo salvador',
 'clara fernanda fragoso islas',
 'francisco',
 'max ramirez',
 'jesus ismael pineda hernandez',
 'danyaell martinez ortiz',
 'carolina honorato garcia',
 'esmeralda nava',
 'celeste tello',
 'diana sandoval herna

In [ ]:
ABC = grup['Nombre'].apply(lambda x: unidecode.unidecode(str(x).lower()))
ABC_TF = ABC.isin([a for a, b in registrado])

# Creación de grupos

In [ ]:
# https://docs.google.com/spreadsheets/d/1f7SOx1I39lIxUvzGuX4XIF58g_3H7kfQeYDhew3H4vA/edit#gid=0

In [ ]:
import pandas as pd
from math import ceil
from google.colab import files
enc_path = '/content/drive/MyDrive/Resultados Encuesta.csv'

In [ ]:
grup = pd.read_csv(enc_path)

In [ ]:
grup = grup[ABC_TF]

In [ ]:
g_size = 3
est_size = len(grup)

In [ ]:
clust_g = ['CL Conocimientos', 'CL Gustos']
dist_grup = grup[clust_g].value_counts().reset_index()
estudiantes = grup['Nombre']

In [ ]:
con_size = len(grup['CL Conocimientos'].unique())
n_grupos = ceil(est_size/g_size)

Grupos = {i:[] for i in range(n_grupos)}
grup_c = grup.copy()

n_g = 0

#Grupos ideales
for i in range(con_size):
  agg_grup = dist_grup.iloc[i*con_size:(i+1)*con_size,]
  grupos_justos = min(agg_grup[0])
  for veces in range(grupos_justos):
    # Siempre se hará un número de veces igual al número de clusters de gustos distintos (3)
    for con, gts in agg_grup.drop(columns = [0]).values:
      estudiantes = grup_c[(grup_c['CL Conocimientos']== con)&(grup_c['CL Gustos']== gts)]
      estudiantes = estudiantes.sample(1)
      grup_c.drop(estudiantes.index, inplace = True)
      Grupos[n_g].append(list(estudiantes[['Nombre','Correo electrónico']].values[0]))
    n_g +=1

extras = grup_c[clust_g].value_counts().reset_index()
grupos_ext = extras[extras[0]>=g_size]

#Grupos imperfectos agruados por los gustos

for con, gts, n_gp in grupos_ext.values:
  # Siempre se hará un número de veces igual al número de clusters de gustos distintos (3)

  estudiantes = grup_c[(grup_c['CL Conocimientos']== con)&(grup_c['CL Gustos']== gts)]

  for n_aas in range(int(n_gp/g_size)):
    estudiantes_ = estudiantes.sample((n_aas+1)*g_size)

    grup_c.drop(estudiantes_.index, inplace = True)
    Grupos[n_g] = [[n,c]for n, c in estudiantes_[['Nombre',	'Correo electrónico']].values]
    n_g +=1
  
# Grupos de los que "no encajaron"

for n_aas in range(ceil(grup_c.shape[0]/g_size)):
  estudiantes_ = grup_c.sample(min(grup_c.shape[0],g_size))
  grup_c.drop(estudiantes_.index, inplace = True)
  Grupos[n_g] = [[n,c]for n, c in estudiantes_[['Nombre',	'Correo electrónico']].values]
  n_g +=1

In [ ]:
grupos_ord = []
for nom in grup['Nombre']:
  grupos_ord.append([Grupos[g] for g in Grupos.keys() if any([nom == l[0] for l in Grupos[g]])][0])

grup['Grupo'] = grupos_ord

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
grup.to_csv('Resultados Encuesta.csv', index = False, encoding='utf-8-sig')
files.download('Resultados Encuesta.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>